In [171]:
import pandas as pd
import os
import ast
import re

!pip install python-Levenshtein
import Levenshtein


The folder you are executing pip from can no longer be found.


In [172]:
def extract_text(text):
    try:
        dict_obj = ast.literal_eval(text)
        return dict_obj['text']
    except (ValueError, KeyError):
        return pd.NA


In [173]:
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]','',text) #removing special character and digits
    text = text.lower()
    return text

In [174]:
def merge_and_preprocess_df(path1,path2):
    tran_df = pd.read_csv(path1)
    tran_df.drop(columns=['Subset_Name'], inplace=True)
    tran_df = tran_df.rename(columns={'File': 'file_name'})
    
    metadata_df = pd.read_csv(path2)
    metadata_df = metadata_df[['file_name','phrase','merged_prompt']]
    metadata_df.head()
    

    final_df = pd.merge(tran_df, metadata_df, on='file_name', how='inner')
    final_df['Transcription'] = final_df['Transcription'].apply(extract_text)
    final_df['Transcription'] = final_df['Transcription'].apply(preprocess_text)
    return final_df

In [175]:
metadata_path = '../../../data/metadata'
print(os.listdir(train_metadata_path))

meta_train = '../../../data/metadata/metadata_train.csv'
meta_test = '../../../data/metadata/metadata_test.csv'
meta_validate = '../../../data/metadata/metadata_validate.csv'

['metadata_train.csv', 'metadata_test.csv', 'metadata_validate.csv']


In [176]:
train_path = 'transcription_train_whisper-ai.csv'
test_path = 'transcriptions_test_whisper-ai.csv'
validate_path = 'transcriptions_validate_whisper-ai.csv'

In [177]:
transcription_train = merge_and_preprocess_df(train_path,meta_train)
transcription_train.head()

FileNotFoundError: [Errno 2] No such file or directory: 'transcription_train_whisper-ai.csv'

In [ ]:
transcription_validate = merge_and_preprocess_df(validate_path,meta_validate)
transcription_validate.head()

In [ ]:
transcription_test = merge_and_preprocess_df(test_path,meta_test)
transcription_test.head()

In [ ]:
def wer(reference, hypothesis):
    """
    Calculate Word Error Rate (WER) between reference and hypothesis.
    """
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    # Calculate Levenshtein distance
    distance = Levenshtein.distance(reference_words, hypothesis_words)

    # Calculate WER
    wer = distance / len(reference_words)
    return round(wer,4)


def cer(reference, hypothesis):
    """
    Calculate Character Error Rate (CER) between reference and hypothesis.
    """
    # Calculate Levenshtein distance
    distance = Levenshtein.distance(reference, hypothesis)

    # Calculate CER
    cer = distance / len(reference)
    return round(cer,4)

In [ ]:
wer_scores = []
cer_scores = []

for actual, pred in zip(transcription_train['phrase'], transcription_train['Transcription']):
    wer_scores.append(wer(actual, pred))
    cer_scores.append(cer(actual, pred))

In [ ]:
transcription_train['word_error_rate'] = wer_scores
transcription_train['character_error_rate'] = cer_scores

In [ ]:
transcription_train

In [ ]:
print('Mean WER:', round(transcription_train['word_error_rate'].mean(), 4))

In [ ]:
print('Mean CER:', round(transcription_train['character_error_rate'].mean(), 4))

In [ ]:
# transcription_train.to_csv('/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/whisper-ai/train_recordings_transcription_whisper-ai.csv', index=False)

## TEST

In [ ]:
wer_scores = []
cer_scores = []

for actual, pred in zip(transcription_test['phrase'], transcription_test['Transcription']):
    wer_scores.append(wer(actual, pred))
    cer_scores.append(cer(actual, pred))

transcription_test['word_error_rate'] = wer_scores
transcription_test['character_error_rate'] = cer_scores


print('Mean WER:', round(transcription_test['word_error_rate'].mean(), 4))

print('Mean CER:', round(transcription_test['character_error_rate'].mean(), 4))

# recordings_df.to_csv('./train_recordings_transcription_AWS.csv', index=False)

In [ ]:
# transcription_test.to_csv('/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/whisper-ai/test_recordings_transcription_whisper-ai.csv', index=False)

## Validate

In [ ]:
wer_scores = []
cer_scores = []

for actual, pred in zip(transcription_validate['phrase'], transcription_validate['Transcription']):
    wer_scores.append(wer(actual, pred))
    cer_scores.append(cer(actual, pred))

transcription_validate['word_error_rate'] = wer_scores
transcription_validate['character_error_rate'] = cer_scores


print('Mean WER:', round(transcription_validate['word_error_rate'].mean(), 4))

print('Mean CER:', round(transcription_validate['character_error_rate'].mean(), 4))


In [ ]:
transcription_validate

In [ ]:
# transcription_validate.to_csv('/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/whisper-ai/validate_recordings_transcription_whisper-ai.csv', index=False)

## Comparing AWS and Whisper-AI

In [178]:
aws_file_path = "/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/aws/train_recordings_transcription_AWS.csv"
aws_df = pd.read_csv(aws_file_path)
aws_df.head()

,file_name,actual_transcription,pred_transcription,word_error_rate,character_error_rate
0,1249120_13842059_104469105.wav,I have a painful cramp in my feet,I have a painful cramp in my feet.,0.1250,0.0303
1,1249120_13842059_105045085.wav,The pain feels like it's right below the skin,The pain feels like it's right below the skin.,0.1111,0.0222
2,1249120_13842059_11964685.wav,I feel suicidal.,I feel suicidal.,0.0000,0.0000
3,1249120_13842059_12420758.wav,I feel a sharp pain in my ankle joint when I s...,I feel a sharp pain in my ankle joint when I s...,0.0000,0.0000
4,1249120_13842059_13041979.wav,My shoulder hurts me so much,My shoulder hurts me so much.,0.1667,0.0357


In [186]:
print('Mean WER:', round(aws_df['word_error_rate'].mean(), 4))

print('Mean CER:', round(aws_df['character_error_rate'].mean(), 4))

Mean WER: 0.181
Mean CER: 0.0467


In [184]:
whisper_file_path = "/Users/kunalindore/Library/CloudStorage/OneDrive-NortheasternUniversity/Capstone/Multi-Modal-Intent-Recognition-in-Healthcare/audio_transcription/transcription_files/whisper-ai/wer_cer_scores/train_recordings_transcription_whisper-ai.csv"
whisper_df = pd.read_csv(whisper_file_path)
whisper_df = whisper_df.rename(columns={'Transcription': 'pred_transcription', 'phrase': 'actual_transcription'})
whisper_df.head()

,file_name,pred_transcription,actual_transcription,merged_prompt,word_error_rate,character_error_rate
0,1249120_15004831_103229689.wav,my son had his lip pierced and it is swollen ...,my son had his lip pierced and it is swollen a...,Wound and injury,0.0455,0.0196
1,1249120_15004831_93767590.wav,my back hurts so much i cant bend down to tie...,my back hurts so much i cant bend down to tie ...,"Neck, back or spinal issues",0.0000,0.0172
2,1249120_15830408_55921206.wav,i was travelling by ship and i feel dizzy,i was travelling by ship and i feel dizzy,Dizziness and vertigo,0.0000,0.0244
3,1249120_15830408_37803938.wav,i do not know why i feel cold,i do not know why i feel cold,Feeling cold/hot,0.0000,0.0345
4,1249120_2171654_19438090.wav,my hair is falling out after i take a shower,my hair is falling out after i take a shower,Hair and skin issues,0.0000,0.0227


In [185]:
print('Mean WER:', round(whisper_df['word_error_rate'].mean(), 4))

print('Mean CER:', round(whisper_df['character_error_rate'].mean(), 4))

Mean WER: 0.0413
Mean CER: 0.0406
